In [12]:
import ifcopenshell
import ifcopenshell.geom as geom

model = ifcopenshell.open("F:/IFC-Rail-Unit-Test-Reference-Code/alignment_testset/IFC-WithGeneratedGeometry/GENERATED__VerticalAlignment_ParabolicArc_100.0_10.0_0.0_0.5_1_Meter.ifc")

inst = model.by_type('IfcGradientCurve')[0]
pwf = ifcopenshell.ifcopenshell_wrapper.map_shape(ifcopenshell.geom.settings(),inst.wrapped_data)
evaluator = ifcopenshell.ifcopenshell_wrapper.piecewise_function_evaluator(pwf,ifcopenshell.geom.settings())

print(pwf.length())
print(pwf.spans())



100.0
<Swig Object of type 'ifcopenshell::geometry::taxonomy::piecewise_function::spans_t *' at 0x00000177D50E6A90>
